<a id='index'></a>
# Single Site Multi Suite Model Example

- [Base case](#base_case) - original Example 1 from Lakhdar et al. (2005)
- [Increased demand for product p1](#increased_p1_demand)
- [Doubled demand profile](#doubled_demand_profile)

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

import plotly.offline as opy
import plotly.graph_objs as go
import plotly.figure_factory as ff

from IPython.core.display import display, HTML

from biopharma_scheduling_ESCAPE27.scheduling_models import Example1Model

In [ ]:
opy.init_notebook_mode()

In [ ]:
def make_demand_supply_chart(sales_profile, demand_data, title):
    x = ['t%d' % i for i in range(1, len(demand_data) + 1)]

    trace1 = go.Bar(
        x=x,
        y=demand_data,
        name='Demand',
        opacity=0.6
    )

    trace2 = go.Bar(
        x=x,
        y=sales_profile,
        name='Supply',
        opacity=0.6
    )

    layout = go.Layout(
        title=title,
        yaxis=dict(
            title='No. batches'),
        xaxis=dict(
            title='Time period')
    )

    return opy.iplot(go.Figure(data=[trace1, trace2], layout=layout))

In [ ]:
colors = ['rgb(244, 66, 66)', 'rgb(65, 244, 92)', 'rgb(73, 65, 244)']

def make_gantt(schedule, title, colors=colors, start_date='2017-01-01'):        
    df = schedule.reset_index()
    df = df[df['Resource'] != 'Product p0']
    
    for col in ['Start', 'Finish']:
        df[col] = df[col].apply(lambda x: (pd.Timedelta('%d days' % x) + pd.to_datetime(start_date)).date())
        
    df = df.to_dict('records')
    
    gantt = ff.create_gantt(df, colors=colors, showgrid_x=True, index_col='Resource', group_tasks=True)

    gantt['layout'].update({'title': ''})
    gantt['layout'].update({'width': 800})
    gantt['layout'].update({'height': 500})
    gantt['layout'].update({
        'xaxis': {
            'tickangle': -30,
            'side': 'top'
        }
    })
    
    return opy.iplot(
        gantt,
        config=dict(
            displaylogo=False,
            modeBarButtonsToRemove=['sendDataToCloud']
        )
    )

<a id='base_case'></a>
## Base case
A continuous-time, genetic algorithm based solution to the original **Example 1 problem** from *"[Medium term planning of biopharmaceutical manufacture using mathematical programming. Biotechnology progress](http://onlinelibrary.wiley.com/doi/10.1021/bp0501571/full)"* by Lakhdar et al. (2005).

[back to index](#index)

In [ ]:
num_usp_suites = 2
num_dsp_suites = 2

print("No. USP Suites: %d, no. DSP Suites: %d" % (num_usp_suites, num_dsp_suites))

In [ ]:
product_data = pd.read_csv('product_data.csv', index_col='Product')
product_data

In [ ]:
days_per_period = [60, 60, 60, 60, 60, 60]
demand_data = pd.read_csv('demand_data.csv', index_col='Product')

In [ ]:
example1_model = Example1Model().fit(
    num_usp_suites,
    num_dsp_suites,
    
    demand_data.values,
    days_per_period,
    
    product_data['USP storage cost'],
    product_data['Sales price'],
    product_data['Prod. cost'],
    product_data['Waste disp. cost'],
    product_data['DSP storage cost'],
    product_data['Backlog penalty'],
    product_data['Changeover cost'],
    
    product_data['USP rate [days]'],
    product_data['USP lead [days]'],
    product_data['USP lifetime [days]'],
    product_data['USP cap. per 60 days'],
    
    product_data['DSP rate [days]'],
    product_data['DSP lead [days]'],
    product_data['DSP lifetime [days]'],
    product_data['DSP cap. per 60 days']
)

In [ ]:
df = pd.DataFrame({key : [value] for key, value in example1_model.objectives.items()}, index=['value'])
df[['profit', 'sales', 'backlog_cost', 'production_cost', 'changeover_cost', 'dsp_storage_cost', 'dsp_waste_cost']]

In [ ]:
example1_model.schedule

In [ ]:
make_gantt(example1_model.schedule, 'Base case')

In [ ]:
make_demand_supply_chart(example1_model.sales_profile[0], demand_data.values[0], 'Product p1 Demand & Supply')

In [ ]:
make_demand_supply_chart(example1_model.sales_profile[1], demand_data.values[1], 'Product p2 Demand & Supply')

In [ ]:
make_demand_supply_chart(example1_model.sales_profile[2], demand_data.values[2], 'Product p3 Demand & Supply')

<a id='increased_p1_demand'></a>
## Increased demand for product p1
Demand for p1 at time period t6 is increased by 3 batches.

[back to index](#index)

In [ ]:
increased_p1_demand = demand_data.copy().values
increased_p1_demand[0][-1] = 9
increased_p1_demand = pd.DataFrame(increased_p1_demand.tolist(), columns=demand_data.columns, index=demand_data.index)

In [ ]:
example1_model = Example1Model().fit(
    num_usp_suites,
    num_dsp_suites,
    
    increased_p1_demand.values,
    days_per_period,
    
    product_data['USP storage cost'],
    product_data['Sales price'],
    product_data['Prod. cost'],
    product_data['Waste disp. cost'],
    product_data['DSP storage cost'],
    product_data['Backlog penalty'],
    product_data['Changeover cost'],
    
    product_data['USP rate [days]'],
    product_data['USP lead [days]'],
    product_data['USP lifetime [days]'],
    product_data['USP cap. per 60 days'],
    
    product_data['DSP rate [days]'],
    product_data['DSP lead [days]'],
    product_data['DSP lifetime [days]'],
    product_data['DSP cap. per 60 days']
)

In [ ]:
df = pd.DataFrame({key : [value] for key, value in example1_model.objectives.items()}, index=['value'])
df[['profit', 'sales', 'backlog_cost', 'production_cost', 'changeover_cost', 'dsp_storage_cost', 'dsp_waste_cost']]

In [ ]:
make_gantt(example1_model.schedule, "Increased demand for p1")

In [ ]:
make_demand_supply_chart(example1_model.sales_profile[0], increased_p1_demand.values[0], 'Product p1 Demand & Supply')

In [ ]:
make_demand_supply_chart(example1_model.sales_profile[1], increased_p1_demand.values[1], 'Product p2 Demand & Supply')

In [ ]:
make_demand_supply_chart(example1_model.sales_profile[2], increased_p1_demand.values[2], 'Product p3 Demand & Supply')

<a id='doubled_demand_profile'></a>
## Doubled demand profile
The demand profile from the original Example 1 problem is repeated for another year.

[back to index](#index)

In [ ]:
import numpy as np

columns = ['t%d' % _ for _  in range(1, 13)]
doubled_demand_profile = pd.DataFrame(np.hstack([demand_data.values, demand_data.values]).tolist(), 
                                      columns=columns, 
                                      index=demand_data.index)

days_per_period_doubled = days_per_period + days_per_period

In [ ]:
example1_model = Example1Model().fit(
    num_usp_suites,
    num_dsp_suites,
    
    doubled_demand_profile.values,
    days_per_period_doubled,
    
    product_data['USP storage cost'],
    product_data['Sales price'],
    product_data['Prod. cost'],
    product_data['Waste disp. cost'],
    product_data['DSP storage cost'],
    product_data['Backlog penalty'],
    product_data['Changeover cost'],
    
    product_data['USP rate [days]'],
    product_data['USP lead [days]'],
    product_data['USP lifetime [days]'],
    product_data['USP cap. per 60 days'],
    
    product_data['DSP rate [days]'],
    product_data['DSP lead [days]'],
    product_data['DSP lifetime [days]'],
    product_data['DSP cap. per 60 days']
)

In [ ]:
make_gantt(example1_model.schedule, "Doubled demand profile")

In [ ]:
df = pd.DataFrame({key : [value] for key, value in example1_model.objectives.items()}, index=['value'])
df[['profit', 'sales', 'backlog_cost', 'production_cost', 'changeover_cost', 'dsp_storage_cost', 'dsp_waste_cost']]

In [ ]:
make_demand_supply_chart(example1_model.sales_profile[0], doubled_demand_profile.values[0], 'Product p1 Demand & Supply')

In [ ]:
make_demand_supply_chart(example1_model.sales_profile[1], doubled_demand_profile.values[1], 'Product p2 Demand & Supply')

In [ ]:
make_demand_supply_chart(example1_model.sales_profile[2], doubled_demand_profile.values[2], 'Product p3 Demand & Supply')

[back to index](#index)